In [ ]:
from ParseData import K, GameMode, Side
from ParsePosition import parse_positions
import altair as alt
import pandas as pd

def generate_games_played_dataset(data, brush):   
    bars = alt.Chart().mark_bar().encode(
        x=alt.X('month:O', axis=alt.Axis(title='Months')),
        y=alt.Y('count()', axis=alt.Axis(title='Games played')),
        opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.5)),
    )
    
    background = bars.properties(selection=brush)
    
    highlight = bars.encode(color=alt.value('goldenrod'), opacity=alt.OpacityValue(1)).transform_filter(brush)
    
    barsZ = alt.Chart().mark_bar().encode(
        x=alt.X('month:O', axis=alt.Axis(title='Months')),
        y='count()',
    ).transform_filter(
        brush
    )
    
    lineZ = alt.Chart().mark_rule(color='firebrick').encode(
        y=alt.Y('mean(count_games):Q', axis=alt.Axis(title="Mean of games played")),
        size=alt.SizeValue(3)
    ).transform_aggregate(
        count_games='count()',
        groupby=["month"]
    ).transform_filter(
        brush
    )


    graph = alt.hconcat(alt.layer(background, highlight, data=data), alt.layer(barsZ, lineZ, data= data))
    return graph

def generate_game_mode_histogram(data, brush):
    base = alt.Chart().mark_bar().encode(
        x=alt.X(K.gameMode.value + ':N', axis=alt.Axis(title="GameMode")),
        y=alt.X('count()', axis=alt.Axis(title="Games played"))
        # opacity=alt.condition(sbrush, alt.OpacityValue(1), alt.OpacityValue(0.5))
    )
    
    background = base.properties(selection=brush)
    
    highlight = base.encode(color=alt.value('goldenrod')).transform_filter(brush)
        
    return alt.layer(background, highlight, data=data)

def generate_win_ratio_graph(data, brush):
    base = alt.Chart().mark_line().encode(
        x=alt.X('month:O', axis=alt.Axis(title='Months')),
        y=alt.Y('win_ratio:Q'.format(gameDuration=K.gameDuration.value), axis=alt.Axis(title="Win ratio")),
        opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.5)),
    )
    
    background = base.transform_aggregate(
        wins='sum({win})'.format(win=K.win.value),
        count_games='count()',
        groupby=["month"]
    ).transform_calculate(
        win_ratio='datum.wins/datum.count_games'
    )
    
    highlight = base.encode(color=alt.value('goldenrod'), opacity=alt.OpacityValue(1)).transform_filter(brush).transform_aggregate(
        wins='sum({win})'.format(win=K.win.value),
        count_games='count()',
        groupby=["month"]
    ).transform_calculate(
        win_ratio='datum.wins/datum.count_games'
    )
    
    line = alt.Chart().mark_rule(color='firebrick').encode(
        y=alt.Y('mean(win_ratio):Q'.format(gameDuration=K.gameDuration.value), axis=alt.Axis("Mean of Win Ratio")),
    ).transform_filter(brush).transform_aggregate(
        wins='sum({win})'.format(win=K.win.value),
        count_games='count()'
    ).transform_calculate(
        win_ratio='datum.wins/datum.count_games'
    )
    
    return alt.layer(background, highlight, line, data=data)

def generate_champ_bar_histogram(data, brush):
    base = alt.Chart().mark_bar().encode(
        x=alt.X('{champion}:N'.format(champion=K.championId.value), axis=alt.Axis(title="Champion")),
        y=alt.Y('champ_count:Q', axis=alt.Axis(title="Games played")),
        opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.5)),
    )
    
    background = base.properties(selection=brush).transform_aggregate(
        champ_count='count()',
        groupby=[K.championId.value]
    )
    
    highlight = base.encode(color=alt.value('goldenrod'), opacity=alt.OpacityValue(1)).transform_filter(brush).transform_aggregate(
        champ_count='count()',
        groupby=[K.championId.value]
    )

    return alt.layer(background, highlight, data=data)


def generate_game_duration_histogram(data, brush):
    base = alt.Chart().mark_line().encode(
        x=alt.X('month:O', axis=alt.Axis(title='Months')),
        y=alt.Y('mean({gameDuration}):Q'.format(gameDuration=K.gameDuration.value), axis=alt.Axis(title="Mean of Game Duration")),
        opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.5))
    )
    
    highlight = base.encode(color=alt.value('goldenrod'), opacity=alt.OpacityValue(1)).transform_filter(brush)

    
    lineZ = alt.Chart().mark_rule(color='firebrick').encode(
        y='mean({gameDuration}):Q'.format(gameDuration=K.gameDuration.value),
        size=alt.SizeValue(3)
    ).transform_filter(brush)
    
    max = generate_label("Maximo", 'max({gameDuration}):Q'.format(gameDuration = K.gameDuration.value), brush)
    min = generate_label("Minimo", 'min({gameDuration}):Q'.format(gameDuration = K.gameDuration.value), brush)
    total = generate_label("Total", 'sum({gameDuration}):Q'.format(gameDuration = K.gameDuration.value), brush)
    
    
    return alt.hconcat(alt.layer(base, highlight, lineZ, data=data), v_concat_rec([total, max, min], data))


def generate_position_champ_bar(data, brush):
    base = alt.Chart().mark_bar().encode(
        x=alt.X('{champion}:N'.format(champion=K.championId.value), axis=alt.Axis(title="Champion")),
        y=alt.Y('champ_count:Q', axis=alt.Axis(title="Games played")),
        opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.5)),
    )
    
    background = base.properties(selection=brush).transform_aggregate(
        champ_count='distinct({champ})'.format(champ=K.gameId.value),
        groupby=[K.championId.value]
    )
    
    highlight = base.encode(color=alt.value('goldenrod'), opacity=alt.OpacityValue(1)).transform_filter(
        brush
    ).transform_aggregate(
        champ_count='distinct({champ})'.format(champ=K.gameId.value),
        groupby=[K.championId.value]
    )

    return alt.layer(background, highlight, data=data)

def generate_position_map(data, brush):
    classic = data.loc[data[K.gameMode.value] == GameMode.CLASSIC.value]
    blue = classic.loc[classic[K.teamId.value] == Side.BLUE.value]
    red = classic.loc[classic[K.teamId.value] == Side.RED.value]
    dataBlue=classic.loc[classic[K.gameId.value].isin(blue[K.gameId.value])]
    dataRed=classic.loc[classic[K.gameId.value].isin(red[K.gameId.value])]
    
    
    pointsB = alt.Chart(dataBlue).mark_circle().encode(
        x='x:Q',
        y='y:Q',
        size=alt.value(10),
        color=alt.value('blue')
    ).transform_filter(brush)
    
    pointsR = alt.Chart(dataRed).mark_circle().encode(
        x='x:Q',
        y='y:Q',
        size=alt.value(10),
        color=alt.value('red')
    ).transform_filter(brush)
    
    hData = data.loc[data['x'] > 1100]
    hData = hData.loc[hData['x'] < 13800]
    hData = hData.loc[hData['y'] > 1100]
    hData = hData.loc[hData['y'] < 23400]
    heatmap = alt.Chart(hData[0:4999]).mark_rect().encode(
        alt.X('x:Q', bin=alt.Bin(maxbins=100)),
        alt.Y('y:Q', bin=alt.Bin(maxbins=100)),
        alt.Color('count(x) + count(y):Q', scale=alt.Scale(scheme='greenblue'))
    ).transform_filter(brush)
    
    return alt.layer(pointsB, pointsR), heatmap    

def generate_label(title, encode, brush):
    return alt.Chart().mark_text(align='center', baseline='middle').encode(
            text=encode
            ).transform_filter(brush).properties(title=title)

def v_concat_rec(charts, data):
    if len(charts) < 1:
        raise Exception("At least 2")
    
    if len(charts) == 1:
        return alt.layer(charts[0], data=data)
    
    return alt.vconcat(v_concat_rec(charts[:-1], data), charts[-1], data=data)
    
    

def generate_win_ratio_histogram(data, brush):
    base = alt.Chart().mark_bar().encode(
        x=K.gameMode.value + ':N',
        y='count()'
    )
    
    
    
    
    
    